# Q1. Donwloading the data

In [2]:
import pandas as pd
import pickle
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

dv = DictVectorizer()

In [3]:
df_january = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

In [3]:
df_january.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [4]:
print("Number of columns: " + str(df_january.columns.size))

Number of columns: 19


# There are 19 columns in total

# Q2. Computing duration

In [17]:
df_january.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [4]:
df_january_pickup_datetime = df_january['tpep_pickup_datetime']
df_january_dropoff_datetime = df_january['tpep_dropoff_datetime']

In [5]:
type(df_january_pickup_datetime)
df_january_duration = df_january_dropoff_datetime -  df_january_pickup_datetime

In [8]:
df_january_duration.head()

0   0 days 00:08:26
1   0 days 00:06:19
2   0 days 00:12:45
3   0 days 00:09:37
4   0 days 00:10:50
dtype: timedelta64[ns]

# Duration in minutes

In [6]:
df_january_min = df_january_duration / np.timedelta64(1, 'm')
df_january_min

0           8.433333
1           6.316667
2          12.750000
3           9.616667
4          10.833333
             ...    
3066761    13.983333
3066762    19.450000
3066763    24.516667
3066764    13.000000
3066765    14.400000
Length: 3066766, dtype: float64

In [6]:
standard_deviation = np.std(df_january_min)
print("Standard desviation is: " + str(standard_deviation))

Standard desviation is: 42.5943442974141


## The standard desviation is: 42.59

## Q3. Dropping outliers

In [7]:
df_january['duration'] = df_january_min
df_jan_min_u_60 = df_january[(df_january['duration'] >= 1) & (df_january['duration'] <= 60)].copy()
df_jan_min_u_60

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.30,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.90,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.90,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.10,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.40,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066761,2,2023-01-31 23:58:34,2023-02-01 00:12:33,NaN,3.05,NaN,None,107,48,0,15.80,0.00,0.5,3.96,0.0,1.0,23.76,NaN,NaN,13.983333
3066762,2,2023-01-31 23:31:09,2023-01-31 23:50:36,NaN,5.80,NaN,None,112,75,0,22.43,0.00,0.5,2.64,0.0,1.0,29.07,NaN,NaN,19.450000
3066763,2,2023-01-31 23:01:05,2023-01-31 23:25:36,NaN,4.67,NaN,None,114,239,0,17.61,0.00,0.5,5.32,0.0,1.0,26.93,NaN,NaN,24.516667
3066764,2,2023-01-31 23:40:00,2023-01-31 23:53:00,NaN,3.15,NaN,None,230,79,0,18.15,0.00,0.5,4.43,0.0,1.0,26.58,NaN,NaN,13.000000


In [12]:
fraction = (df_jan_min_u_60['duration'].size / df_january_min.size ) * 100
print("Fraction of the records left: " + str(int(fraction)) + "%")

Fraction of the records left: 98%


## Q4. One-hot encoding

In [8]:
categorical = ['PULocationID', 'DOLocationID']

df_jan_min_u_60['PULocationID'] = df_jan_min_u_60['PULocationID'].astype(str)
df_jan_min_u_60['DOLocationID'] = df_jan_min_u_60['DOLocationID'].astype(str)

df_dict = df_jan_min_u_60[categorical].to_dict(orient='records')
df_fit = dv.fit_transform(df_dict)

In [11]:
len(dv.get_feature_names_out())

515

In [14]:
print("Dimensionality of this matrix (number of columns) is: " + str(len(dv.get_feature_names_out())))

Dimensionality of this matrix (number of columns) is: 515


## Q5. Training a model

In [8]:
reg = linear_model.LinearRegression()

x_train = df_fit

y_train = df_jan_min_u_60['duration']

reg.fit(x_train, y_train)

LinearRegression()

In [16]:
y_pred_train = reg.predict(x_train)
rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
rmse_train

7.649261027792376

In [17]:
print("RMSE on train is: " + str(rmse_train))

RMSE on train is: 7.649261027792376


## Saving the model because it causes the kernel to stop and need to reset variables in Q6

In [9]:
with open('models/lin_reg_hw.bin', 'wb') as f_out:
    pickle.dump((dv, reg), f_out)

## Q6. Evaluating the model

In [2]:
df_february = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [3]:
df_february_pickup_datetime = df_february['tpep_pickup_datetime']
df_february_dropoff_datetime = df_february['tpep_dropoff_datetime']
df_february_duration = df_february_dropoff_datetime -  df_february_pickup_datetime
df_february_min = df_february_duration / np.timedelta64(1, 'm')

In [4]:
df_february['duration'] = df_february_min
df_feb_min_u_60 = df_february[(df_february['duration'] >= 1) & (df_february['duration'] <= 60)].copy()
df_feb_min_u_60['PULocationID'] = df_feb_min_u_60['PULocationID'].astype(str)
df_feb_min_u_60['DOLocationID'] = df_feb_min_u_60['DOLocationID'].astype(str)

In [5]:
categorical = ['PULocationID', 'DOLocationID']
df_feb_dict = df_feb_min_u_60[categorical].to_dict(orient='records')

In [6]:
with open("models/lin_reg_hw.bin", "rb") as f:
    dv, model = pickle.load(f)

In [7]:
df_feb_fit = dv.transform(df_feb_dict)

In [8]:
x_feb_val = df_feb_fit
y_val = df_feb_min_u_60['duration']
y_pred_val = model.predict(x_feb_val)
rmse_train = np.sqrt(mean_squared_error(y_val, y_pred_val))
rmse_train

7.811832836304415

In [9]:
print("The RMSE on validation is: " + str(rmse_train))

The RMSE on validation is: 7.811832836304415
